In [1]:
import configparser
import os
import imaplib
import email
import firebase_admin
from firebase_admin import credentials, firestore

In [9]:
def get_config_data(iten_title, iten, config_path=os.path.join(os.getcwd(), 'config.txt')):
    """
    It reads a config file and returns the value of a given item
    :param iten_title: The title of the section in the config file
    :param iten: the name of the parameter you want to get from the config file
    :param config_path: The path to the config file
    :return: A string
    """

    arq_config = configparser.RawConfigParser()
    arq_config.read(config_path)

    data = arq_config.get(iten_title, iten)

    return str(data)

def create_email_server(usr, psw):
    """
    It creates a server that can send emails
    :return: The server object
    """

    email_server = imaplib.IMAP4_SSL('imap.gmail.com')
    email_server.login(usr, psw)

    return email_server

In [10]:
user = get_config_data(iten_title='EMAIL_LOGIN', iten='email')
psw = get_config_data(iten_title='EMAIL_LOGIN', iten='password')

email_server = create_email_server(user, psw)
email_server.select('EasyMeetingAnswer')

('OK', [b'2'])

In [11]:
status, data = email_server.search(None, 'ALL')
status, data

('OK', [b'1 2'])

In [12]:
mail_ids = [block.split() for block in data].pop(0)
mail_ids

[b'1', b'2']

In [13]:
# define the range for the operation
start = mail_ids[0].decode()
end = mail_ids[-1].decode()

start, end

('1', '2')

In [14]:
status, data = email_server.fetch(mail_ids[0], '(RFC822)')
status, data

('OK',
 [(b'1 (RFC822 {11304}',
   b'Delivered-To: easymeeting.sender@gmail.com\r\nReceived: by 2002:a17:902:ba82:b0:17f:9254:2266 with SMTP id k2csp2774962pls;\r\n        Mon, 17 Oct 2022 15:19:26 -0700 (PDT)\r\nX-Google-Smtp-Source: AMsMyM4JBwEAxprfr/mvfRraQRmmnUXw7kR7wNumzXIp8gLd0StvJjmZIVDhQTFLcIZ8rFrli3r5\r\nX-Received: by 2002:a05:6638:4803:b0:363:c2ad:878 with SMTP id cp3-20020a056638480300b00363c2ad0878mr43411jab.230.1666045166361;\r\n        Mon, 17 Oct 2022 15:19:26 -0700 (PDT)\r\nARC-Seal: i=2; a=rsa-sha256; t=1666045166; cv=pass;\r\n        d=google.com; s=arc-20160816;\r\n        b=RcMCyG69prMXT6DJom/HbSUdmgmONUNItsTn1WXelGpypJUSyBipOmEaEdpgMdFfnR\r\n         PV58/GNSSc49+UL5OqyM8IXieOBBzCIII0xro+6StW0T4Wd+0QYLd7Acf3FJGiMuoda0\r\n         fz/0085rovlLmAj0v6nFxnJIG/XywYmHavr1USMVWcff1xen4iV+BxL1E81UtmYTmwEs\r\n         qt/LkgUfwFUsevYJReh7tm9a6X5qwHgf9NSBtWZqkyJRUEKiQa0LXP6a22CpDuyMjusr\r\n         y1osslPVb0DsfB/14CQgBYaqcRnmEIf/lglVbqU766Y33ZcU/UrM6a5xbYU6nieJPEU5\r\n    

In [18]:
type(data[0])

tuple

In [15]:
for response_part in data:
    # so if its a tuple...
    if isinstance(response_part, tuple):
        # we go for the content at its second element
        # skipping the header at the first and the closing
        # at the third
        message = email.message_from_bytes(response_part[1])

        # with the content we can extract the info about
        # who sent the message and its subject
        mail_from = message['from']
        mail_subject = message['subject']

        # then for the text we have a little more work to do
        # because it can be in plain text or multipart
        # if its not plain text we need to separate the message
        # from its annexes to get the text
        if message.is_multipart():
            mail_content = ''

            # on multipart we have the text message and
            # another things like annex, and html version
            # of the message, in that case we loop through
            # the email payload
            for part in message.get_payload():
                # if the content type is text/plain
                # we extract it
                if part.get_content_type() == 'text/plain':
                    mail_content += part.get_payload()
        else:
            # if the message isn't multipart, just extract it
            mail_content = message.get_payload()

        # and then let's show its result
        print(f'From: {mail_from}')
        print(f'Subject: {mail_subject}')
        print(f'Content: {mail_content}')

From: Vitor Augusto Lima Soares <vitorlima.soares@hotmail.com>
Subject: Teste de email automatico
Content: 
Teste de resposta do email





## Conectando como banco de dados Firestore

In [2]:
cred = credentials.Certificate("FireBase_Project_Key\easyMeeting_prod.json")
firebase_admin.initialize_app(cred)

In [3]:
db = firestore.client() # connecting to firestore

### Insert 
- Criando uma nova collection

In [70]:
user_input = {
  "creator":"Luiz henrique",
  "date": ["29/11/2022", "30/11/2022", "01/12/2022"],
  "email": ["vitorlima.soares@hotmail.com", "vitoraugustodelimasoares@gmail.com"],
  "id":"12",
  "link":"www.zoom.com.br",
  "qtdParticipantes": "2",
  "status":"Pendente",
  "title": "Reuniao de faturamento Novembro 2022"
}

In [71]:
collection = db.collection('meetings')  # create collection
res = collection.document(user_input['id']).set(user_input)
print(res)

update_time {
  seconds: 1669942625
  nanos: 248926000
}



### Find document

In [72]:
collection_name = user_input['id']
collection_name

'12'

In [73]:
res = collection.document(collection_name).get().to_dict()
res

In [89]:
res = collection.get() # returns a list
for i in res: print(i.to_dict())

{'date': ['01/12', '02/12', '05/12'], 'creator': 'Luiz Henrique', 'id': '5a88f296-deb3-4664-973b-7df49e13bc63', 'email_date': [{'email_user': 'luizhgrodrigues@outlook.com', 'date': ''}, {'email_user': 'vitorlima.soares@hotmail.com', 'date': ''}], 'chooseDate': '--/--', 'link': 'https://www.zoom.com/meeting123', 'qtdParticipantes': 2, 'title': 'Reunião de acompanhamento UPX 2022', 'status': 'Pendente', 'email': ['luizhgrodrigues@outlook.com', 'vitorlima.soares@hotmail.com']}


In [ ]:
{"date": ["01/12", "02/12", "05/12"], "creator": "Luiz Henrique", "id": "5a88f296-deb3-4664-973b-7df49e13bc63", "email_date": [{"email_user": "luizhgrodrigues@outlook.com", "date": ""}, {"email_user": "vitorlima.soares@hotmail.com", "date": ""}], "chooseDate": "--/--", "link": "https://www.zoom.com/meeting123", "qtdParticipantes": 2, "title": "Reunião de acompanhamento UPX 2022", "status": "Pendente", "email": ["luizhgrodrigues@outlook.com", "vitorlima.soares@hotmail.com"]}

In [76]:
collection_return = {
    "qtdParticipantes": 3, 
    "creator": "Luiz Henrique", 
    "id": "77caf784-f57b-47d0-bc9d-4bd0761a168e", 
    "chooseDate": "--/--", 
    "email_date": [{"email_user": "luizhgrodrigues@outlook.com", "date": ""}, 
    {"email_user": "alanzinho2012@gmail.com", "date": ""}, 
    {"email_user": "vitorlima@gmail.com", "date": ""}], 
    "link": "https://www.zoom.com/meeting12312", 
    "date": ["10/01", "11/01", "12/01"], 
    "title": "Reunião de finanças Dez 2022", 
    "status": "Pendente", 
    "email": ["luizhgrodrigues@outlook.com", "alanzinho2012@gmail.com", "vitorlima@gmail.com"]
}

In [83]:
data_fb = collection_return['email_date']
data_fb

[{'email_user': 'luizhgrodrigues@outlook.com', 'date': ''},
 {'email_user': 'alanzinho2012@gmail.com', 'date': ''},
 {'email_user': 'vitorlima@gmail.com', 'date': ''}]

In [84]:
data_usuario = 'vitorlima@gmail.com'

In [85]:
for emai_date_dict in data_fb:
    if emai_date_dict['email_user'] == data_usuario:
        emai_date_dict['date'] = "10/01"
        print(emai_date_dict)

{'email_user': 'vitorlima@gmail.com', 'date': '10/01'}


In [87]:
data_fb

[{'email_user': 'luizhgrodrigues@outlook.com', 'date': ''},
 {'email_user': 'alanzinho2012@gmail.com', 'date': ''},
 {'email_user': 'vitorlima@gmail.com', 'date': '10/01'}]

In [ ]:
replace_index = data_fb.index()

### Ler todas as coleções do banco de dados

In [41]:
collection = db.collection('meetings')

In [42]:
res = collection.get() # returns a list
for i in res: print(i.to_dict())

{'date': ['29/11/2022', '30/11/2022', '01/12/2022'], 'qtdParticipantes': 2, 'id': '123456', 'link': 'www.zoom.com.br', 'title': 'Reuniao de faturamen...embro 2022', 'robot_status': 'sucesso', 'status': 'Pendente', 'email': [{'email_user': 'vitoraugustodelimasoares@gmail.com', 'date': '30/11/2022'}, {'email_user': 'vitoraugustodelimaso...@gmail.com', 'date': '01/12/2022'}], 'creator_name': 'Luiz henrique'}
{'qtdParticipantes': 2, 'date': ['29/11/2022', '30/11/2022', '01/12/2022'], 'id': 'd7cc2d8f-28dd-4d81-afed-b56262dfa56f', 'link': 'www.zoom.com.br', 'title': 'Reuniao de faturamen...embro 2022', 'robot_status': 'sucesso', 'email': {'email_user': 'vitoraugustodelimaso...@gmail.com', 'date': '01/12/2022'}, 'status': 'Pendente', 'creator_name': 'Luiz henrique'}
{'date': ['29/11', '30/11', '01/12'], 'creator': 'Luiz Henrique Góes', 'id': 'd7cc2d8f-28dd-4d81-afed-b56262dfa56f', 'chooseDate': '--/--', 'qtdParticipantes': 3, 'link': 'https://www.zoom.com/meetings:129038123', 'title': 'Reuniã

### Buscando o docuimento, filtrando pelo paramentro ID

In [49]:
user_input = {
    'creator_name': 'Luiz henrique', 
    'date': ['29/11/2022', '30/11/2022', '01/12/2022'], 
    'email': {'email_user': 'vitoraugustodelimaso...@gmail.com', 'date': '01/12/2022'}, 
    'id': '9', 
    'link': 'www.zoom.com.br', 
    'qtdParticipantes': 2, 
    'status': 'Pendente', 
    'title': 'Reuniao de faturamen...embro 2022', 
    'robot_status': 'sucesso'}

In [50]:
collection = db.collection('meetings')

In [51]:
collection_list = collection.get()

In [52]:
documnt_list = [doc.to_dict() for doc in collection_list]
documnt_list

[{'qtdParticipantes': 2,
  'date': ['29/11/2022', '30/11/2022', '01/12/2022'],
  'id': '123456',
  'link': 'www.zoom.com.br',
  'title': 'Reuniao de faturamen...embro 2022',
  'robot_status': 'sucesso',
  'status': 'Pendente',
  'email': [{'email_user': 'vitoraugustodelimasoares@gmail.com',
    'date': '30/11/2022'},
   {'email_user': 'vitoraugustodelimaso...@gmail.com', 'date': '01/12/2022'}],
  'creator_name': 'Luiz henrique'},
 {'date': ['29/11/2022', '30/11/2022', '01/12/2022'],
  'creator': 'Luiz henrique',
  'id': '7',
  'qtdParticipantes': '2',
  'link': 'www.zoom.com.br',
  'email_list': ['vitorlima.soares@hotmail.com',
   'vitoraugustodelimasoares@gmail.com'],
  'title': 'Reuniao de faturamento Novembro 2022',
  'status': 'Pendente'},
 {'qtdParticipantes': '2',
  'creator': 'Luiz henrique',
  'id': '8',
  'date': ['29/11/2022', '30/11/2022', '01/12/2022'],
  'link': 'www.zoom.com.br',
  'title': 'Reuniao de faturamento Novembro 2022',
  'email': ['vitorlima.soares@hotmail.com'

In [53]:
collection_name = user_input['id']
collection_name

'9'

In [55]:
main_document = [doc for doc in documnt_list if doc['id']==collection_name][0]
main_document

{'qtdParticipantes': '2',
 'creator': 'Luiz henrique',
 'id': '9',
 'date': ['29/11/2022', '30/11/2022', '01/12/2022'],
 'link': 'www.zoom.com.br',
 'title': 'Reuniao de faturamento Novembro 2022',
 'email': ['vitorlima.soares@hotmail.com',
  'vitoraugustodelimasoares@gmail.com'],
 'status': 'Pendente'}

In [58]:
collection_return = main_document

In [59]:
user_input = {'qtdParticipantes': '2',
 'creator': 'Luiz henrique',
 'id': '9',
 'date': ['29/11/2022', '30/11/2022', '01/12/2022'],
 'link': 'www.zoom.com.br',
 'title': 'Reuniao de faturamento Novembro 2022',
 'email': 'vitorlima.soares@hotmail.com',
 'status': 'Pendente',
 }

In [60]:
data_usuario = user_input['email']
data_fb = collection_return['email']

In [62]:
data_fb

['vitorlima.soares@hotmail.com', 'vitoraugustodelimasoares@gmail.com']

In [63]:
replace_index = data_fb.index(data_usuario)
replace_index

0

In [64]:
data_fb[replace_index] = {"email":data_usuario, "date":"2021"}


In [65]:
data_fb

[{'email': 'vitorlima.soares@hotmail.com', 'date': '2021'},
 'vitoraugustodelimasoares@gmail.com']